In [1]:
import pandas as pd
import dwsemear
import datetime
import collections
import numpy as np

import postgresemear as sql  # mover os arquivos para a pasta principal

def calcula_idade(dados, coluna):
    idade = []
    for i in dados.index:
        if dwsemear.e_nulo(dados.loc[i,coluna]):
            idade.append(None)
        else:
            anos = (dados.loc[i,'datahora'] - dados.loc[i,coluna]).days
            anos = int(round(anos/365))
            idade.append(anos)
    dados['idade'] = idade
    return dados

def remove_localize(data):
    return data.tz_localize(None)

def coluna_para_lista(coluna):
    nova_coluna = []
    for elem in coluna:
        if dwsemear.e_nulo(elem):
            nova_coluna.append(None)
        else:
            nova_coluna.append(elem)
    return nova_coluna

def para_int(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    if type(elemento) not int:
        raise TypeError(str(elemento) + " precisa ser um inteiro")
    else:
        return int(elemento)

def para_float(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    if type(elemento) not float:
        raise TypeError(str(elemento) + " precisa ser um numero real")
    else:
        return float(elemento)

def para_str(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    if type(elemento) not str:
        raise TypeError(str(elemento) + " precisa ser um texto")    
    else:
        return str(elemento)
    
def para_data(elemento):
    if dwsemear.e_nulo(elemento):
        return None
    else:
        return elemento
    
def pegue_elementos_com_repeticao(coluna):
    contagem = collections.Counter(coluna)
    return list(getItemsUpToCount(contagem,1).keys())

def getItemsUpToCount(dct, n):
    upToCount = {}
    for key in dct:
        if dct[key] > n:
            upToCount[key] = dct[key]
    return upToCount


In [28]:
crivo = pd.read_csv('190903_crivo.csv',sep=';', encoding='latin-1',dtype=str)

In [29]:
crivo = dwsemear.limpa_nome_das_colunas(crivo)
crivo = dwsemear.coluna_cpf_para_inteiro(crivo, 'CPF')

In [30]:
# Obtendo timestamp do evento (data + hora)
datahora = []
for i in crivo.index:
    datahora.append(crivo.loc[i,' Data'] + ' ' + crivo.loc[i,' Hora'])
crivo['datahora'] = datahora    
crivo['datahora'] = pd.to_datetime(crivo['datahora'], format='%d/%m/%Y %I:%M:%S %p')

In [36]:
data_inicio = datetime.datetime(2018, 11, 6) # ja fizemos carga dos resultados antes desse
crivo = crivo[crivo.datahora >= data_inicio]
crivo = crivo.reset_index(drop=False)

In [37]:
coluna = 'Data Nascimento Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = pd.to_datetime(crivo[coluna], format='%d/%m/%Y')
crivo = calcula_idade(crivo, coluna) # idade

In [38]:
#coluna = 'Valor Compra Banco Semear Consulta Proposta JRetail PF '
#crivo[coluna] = crivo[coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Renda Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = crivo[coluna].apply(dwsemear.converte_latin_para_float)

In [39]:
coluna = 'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ' # esse e o score?
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Renda presumida Unitfour Consulta perfil PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)

In [40]:
# Obtendo proposta unica (a ultima realizada)
crivo.loc[:,'Codigo proposta'] = crivo.loc[:,'Codigo proposta'].astype(int)
indice_mais_recente = []
for proposta in set(crivo.loc[:,'Codigo proposta']):
    crivo_proposta = crivo[crivo.loc[:,'Codigo proposta'] == proposta]
    ultimo_pedido = crivo_proposta[crivo_proposta.loc[:,'Codigo proposta'] == proposta].datahora.max()
    indice = crivo_proposta[(crivo_proposta.loc[:,'Codigo proposta'] == proposta) &
      (crivo_proposta.loc[:,'datahora'] == ultimo_pedido)].index[0]
    indice_mais_recente.append(indice)
indice_mais_recente.sort()
coluna_mais_recente = np.zeros(len(crivo),dtype=int)
for i in indice_mais_recente:
    coluna_mais_recente[i] = 1
crivo['ultimo_pedido'] = coluna_mais_recente

In [41]:
id_crivo = sql.query_max_coluna('id', 'crivo')
if id_crivo == None:
    id_crivo = 0

In [42]:
crivo_registros = []
for i in crivo.index:
    id_crivo += 1
#    if id_crivo > 10:
#        break

    crivo_registros.append((id_crivo,
          para_int(crivo.loc[i,'Codigo proposta']),
          para_data(crivo.loc[i,'datahora']),  # esse cara poderia ser not null
          para_str(crivo.loc[i,'Discreta UF Banco Semear Consulta Proposta JRetail PF ']),
          para_str(crivo.loc[i,'Discreta Sexo Banco Semear Consulta Proposta JRetail PF ']),
          para_float(crivo.loc[i,'Valor Renda Banco Semear Consulta Proposta JRetail PF ']),
          para_int(crivo.loc[i,'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ']),
          para_float(crivo.loc[i,'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF ']),
          para_float(crivo.loc[i,'Valor Renda presumida Unitfour Consulta perfil PF ']),
          para_float(crivo.loc[i,'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF ']),
          para_int(crivo.loc[i,'ultimo_pedido']),
          para_int(crivo.loc[i,'idade']),  # not null
          para_int(crivo.loc[i,'CPF']),
          para_int(crivo.loc[i,'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF ']),
          para_int(crivo.loc[i,'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF ']),
          para_int(crivo.loc[i,'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '])))
        
#    print('LINHA: ',
#          para_int(crivo.loc[i,'Codigo proposta']),
#          para_data(crivo.loc[i,'datahora']),  # esse cara poderia ser not null
#          para_str(crivo.loc[i,'Discreta UF Banco Semear Consulta Proposta JRetail PF ']),
#          para_str(crivo.loc[i,'Discreta Sexo Banco Semear Consulta Proposta JRetail PF ']),
#          para_float(crivo.loc[i,'Valor Renda Banco Semear Consulta Proposta JRetail PF ']),
#          para_float(crivo.loc[i,'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ']),
#          para_float(crivo.loc[i,'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF ']),
#          para_float(crivo.loc[i,'Valor Renda presumida Unitfour Consulta perfil PF ']),
#          para_float(crivo.loc[i,'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF ']),
#          para_int(crivo.loc[i,'idade']),  # not null
#          para_int(crivo.loc[i,'CPF']),
#          para_int(crivo.loc[i,'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF ']),
#          para_int(crivo.loc[i,'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF ']),
#          para_int(crivo.loc[i,'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '])
#         )
    

In [43]:
query = """ INSERT INTO crivo
            (ID,
            codigo_proposta,
            data,
            uf,
            genero,
            renda_cliente,
            score,
            valor_parcela,
            renda_presumida_unitfour,
            atraso_medio,
            ultimo_pedido,
            idade,
            cpf,
            quantidade_parcelas,
            associados_distintos_30_dias,
            quantidade_consultas) 
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
sql.adicione_multiplos_registros(query, crivo_registros)

# TESTE

In [44]:
parametros = [datetime.datetime(2018,11,1), datetime.datetime(2018,11,30)]
dados_crivo = sql.query_para_pandas('''SELECT *
                      FROM crivo
                      WHERE data BETWEEN %s AND %s
                      ''',parametros)